In [1]:
import json

In [2]:
with open('../dev-v2.0.json', 'r') as inp_reader:
    data = json.load(inp_reader)["data"]

In [3]:
data[0]['paragraphs'][0]['context']

'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

In [4]:
data[0]['paragraphs'][0]['qas']

[{'question': 'In what country is Normandy located?',
  'id': '56ddde6b9a695914005b9628',
  'answers': [{'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159}],
  'is_impossible': False},
 {'question': 'When were the Normans in Normandy?',
  'id': '56ddde6b9a695914005b9629',
  'answers': [{'text': '10th and 11th centuries', 'answer_start': 94},
   {'text': 'in the 10th and 11th centuries', 'answer_start': 87},
   {'text': '10th and 11th centuries', 'answer_start': 94},
   {'text': '10th and 11th centuries', 'answer_start': 94}],
  'is_impossible': False},
 {'question': 'From which countries did the Norse originate?',
  'id': '56ddde6b9a695914005b962a',
  'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Den

### Converting Wefox page xml to SQuad format

In [131]:
import pagexmltools
import os
import pagexml
import pandas as pd

In [132]:
inp_dir = '/Users/nischal/code/wefox/'

In [133]:
pxml = pagexml.PageXML()

In [172]:
def read_merged_xml_get_entities_text(merged_xml):
    pxml.loadXml(merged_xml)
    out_words_entities = []
    entities = []
    for page in pxml.select('//_:Page'):
        page_number = pxml.getPageNumber(page)
        textlines = pxml.select('.//_:TextLine', page)
        idx_textlines = pxml.getLeftRightTopBottomReadingOrder(textlines, fake_baseline=True)
    
        for text_idx in idx_textlines[0]:
            words = pxml.select('_:Word', textlines[text_idx])
            for word in words:
                current_entity = pxml.getPropertyValue(word, 'entity')
                word_entity = {}
                word_entity['word'] = pxml.getTextEquiv(word)
                word_entity['entity'] = current_entity
                word_entity['page'] = page_number
                word_entity['file_name'] = merged_xml
                out_words_entities.append(word_entity)
            
    
            
    
    df = pd.DataFrame(out_words_entities)
    return df
    

In [173]:
df = read_merged_xml_get_entities_text('/Users/nischal/code/wefox/3869e076-29e5-413d-b507-4b1c728584b3/merged.xml')

In [174]:
df.head()

,entity,file_name,page,word
0,insurance_company,/Users/nischal/code/wefox/3869e076-29e5-413d-b...,0,AXA
1,insurance_company,/Users/nischal/code/wefox/3869e076-29e5-413d-b...,0,Versicherung
2,insurance_company,/Users/nischal/code/wefox/3869e076-29e5-413d-b...,0,AG
3,O,/Users/nischal/code/wefox/3869e076-29e5-413d-b...,0,0A/1
4,O,/Users/nischal/code/wefox/3869e076-29e5-413d-b...,0,20050228292


In [214]:
def merge_entity_values(df):
    i = 0
    questions = []
    answers = []
    answer_start_index = []
    while True:
        entity = df.iloc[i]['entity']
        if entity == 'O':
            i += 1
            if i == df.shape[0] - 1:
                break
            continue
        else:
            text_concat = ''
            j = i
            #print("j  %d" %(j,))
            answer_start = j
            while True:
                temp_entity = df.iloc[j]['entity']
                if temp_entity != entity:
                    questions.append("Question is: What is the %s ?" %(entity,))
                    answers.append(text_concat)
                    answer_start_index.append(answer_start)
                    break
                else:
                    text_concat = text_concat + ' ' + df.iloc[j]['word']
                j +=1
            
            i = j
            #print("i is %d"%(i,))
        i +=1
        if i == df.shape[0] - 1:
            break
    
    return questions, answers, answer_start_index
                

In [217]:
def generate_squad_per_page(df):
    page_numbers = df.page.unique()
    for page_number in page_numbers:
        page_df = df.loc[(df.page == page_number)]
        paragraph = page_df.word.str.cat(sep=' ')
        questions, answers, answer_start_index = merge_entity_values(page_df)
        print(page_number)
        print(questions)
        print(answers)
        print(answer_start_index)

In [218]:
generate_squad_per_page(df)

0
['Question is: What is the insurance_company ?', 'Question is: What is the insured_name ?', 'Question is: What is the insured_address ?', 'Question is: What is the insured_address ?', 'Question is: What is the policy_number ?', 'Question is: What is the insured_name ?', 'Question is: What is the insured_address ?', 'Question is: What is the insurance_company ?']
[' AXA Versicherung AG', ' Thomas Hornberger', ' Bismarckstr. 8', ' 31812 Bad Pyrmont', ' 20050228292/0P', ' Thomas Hornberger', ' 8 31812 Bad Pyrmont', ' AXA Versicherung AG']
[0, 23, 31, 35, 50, 165, 168, 173]
1
['Question is: What is the policy_number ?', 'Question is: What is the payment_term ?', 'Question is: What is the product_group ?', 'Question is: What is the end_date ?', 'Question is: What is the premium_display_term ?', 'Question is: What is the premium_gross ?', 'Question is: What is the insurance_company ?', 'Question is: What is the insured_name ?', 'Question is: What is the insured_address ?', 'Question is: Wh

In [146]:
uuid_issues = []
for uuid in os.listdir(inp_dir):
    uuid_path = os.path.join(inp_dir, uuid)
    if os.path.isdir(uuid_path) and uuid != '.git':
        print("Processing directory %s" %(uuid,))
        origin_page_xml = os.path.join(uuid_path, 'origin_page.xml')
        entity_page_xml = os.path.join(uuid_path, 'page.xml')

        if os.path.exists(origin_page_xml)  and os.path.exists(entity_page_xml):
            read_origin_xml_get_words(origin_page_xml)
        else:
            print("Something went wrong for directory %s"%(uuid,))
            uuid_issues.append(uuid)
        break

Processing directory 3869e076-29e5-413d-b507-4b1c728584b3
/Users/nischal/code/wefox/3869e076-29e5-413d-b507-4b1c728584b3/origin_page.xml
None
